In [2]:
import numpy as np
import pandas as pd
from bert_score import score

c:\Users\Asus\Documents\Programming\Research_Assistant\evaluation_radiology_gpt\tf-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('../cleaning_before_eval/ready_to_eval.csv')
df.head()

,No,Question,Dr Answer,Claude Answer,Qwen Answer,GPT Answer,Deepseek RAG Answer,Deepseek non RAG Answer,Claude Full Answer,Qwen Full Answer,GPT Full Answer,Deepseek RAG Full Answer,Deepseek non RAG Full Answer
0,1.0,Pasien mengalami demam pada waktu malam. Sebel...,"Demam Tifoid 70%, Demam Berdarah Dengue 30%","Demam Tifoid, Leptospirosis, Demam Berdarah De...","Demam Biasa, Dispepsia, Asma, Infeksi Saluran ...","Demam Biasa, Demam Tifoid, Demam Berdarah Deng...","Demam Tifoid (Typhoid Fever), Malaria, Leptosp...","Malaria, Infeksi Saluran Kemih (ISK) atau Piel...","Berdasarkan gejala yang disampaikan, saya akan...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan gejala yang dialami pasien, yaitu ...",Berdasarkan gejala yang Anda sebutkan—demam pa...
1,2.0,Pasien mengalami buang air besar cair lebih da...,"Gastroenteritis (GE) Akut 70%, Intoleransi Lak...","Gastroenteritis (GE) Akut, Disentri, Infeksi E...","Diare Akut, Gastroenteritis, Dispepsia, Infeks...","Gastroenteritis Akut, Infeksi Escherichia coli...","Gastroenteritis Akut, Keracunan Makanan, Infek...","Gastroenteritis Akut, Keracunan Makanan, Koler...","Berdasarkan informasi klinis yang diberikan, s...","Berdasarkan gejala yang disampaikan, yaitu bua...","Berdasarkan informasi yang diberikan, pasien m...","Berdasarkan informasi yang diberikan, pasien m...",Berdasarkan gejala yang Anda deskripsikan—diar...
2,3.0,Pasien datang dengan keluhan demam selama 6 ha...,DBD 80%,"Demam Tifoid, Malaria, DBD, Leptospirosis, Inf...","Demam Biasa, Dispepsia, Asma, Demam Dengue, Lu...","Demam Berdarah Dengue, Demam Tifoid, Malaria, ...","Demam Berdarah Dengue (DBD), Demam Tifoid, Lep...","Demam Berdarah Dengue (DBD) / Dengue Fever, Ch...","Berdasarkan informasi yang diberikan, saya aka...","Berdasarkan gejala yang disampaikan, yaitu dem...",Berdasarkan keluhan yang disampaikan oleh pasi...,Berdasarkan keluhan pasien yang meliputi demam...,Berdasarkan gejala yang Anda sebutkan—demam ≥6...
3,4.0,Pasien menderita demam disertai munculnya brun...,"Infeksi Bakteri/Virus (Sistemik) 50%, Infeksi ...","DBD, Infeksi Bakteri/Virus (Sistemik), Infeksi...","Demam Biasa, Dispepsia, Asma, Infeksi Lokal, A...","Infeksi Kulit (Dermatitis atau Selulitis), Kej...","Infeksi Bakteri (Selulitis), Reaksi Alergi (De...",Erisipelas (Infeksi Bakteri pada Kulit dan Jar...,"Berdasarkan informasi klinis yang diberikan, s...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan informasi yang diberikan, pasien m...",### Analisis Gejala dan Kemungkinan Diagnosis\...,Berdasarkan gejala dan temuan pemeriksaan yang...
4,5.0,Pasien nyeri dada sejak 4 jam lalu seperti ter...,"Angina Pekrotis 80%, Infark Miokard Akut (Sera...","Angina Pekrotis, Infark Miokard Akut (Serangan...","Infark Miokard Akut (Serangan Jantung), Dispep...","Angina Pekrotis, Infark Miokard, Penyakit Musk...","Angina Pectoris (Kardiovaskular), Infark Mioka...","Infark Miokard Akut (Serangan Jantung), Angina...",Berdasarkan informasi yang diberikan dalam kon...,"Berdasarkan gejala yang disampaikan, yaitu nye...","Berdasarkan informasi yang diberikan, pasien m...",### Analisis Gejala\n\nPasien mengeluhkan nyer...,Berdasarkan gejala nyeri dada seperti terhimpi...


In [4]:
answer_lists = {}
answer_columns = [col for col in df.columns if col.endswith(' Answer')]

for col_name in answer_columns:
    if (col_name.endswith(' Full Answer')): continue
    model = col_name.removesuffix(' Answer')
    answers = df[col_name].tolist()
    answer_lists[model] = answers

In [5]:
answer_lists

{'Dr': ['Demam Tifoid 70%, Demam Berdarah Dengue 30%',
  'Gastroenteritis (GE) Akut 70%, Intoleransi Laktosa 30%',
  'DBD 80%',
  'Infeksi Bakteri/Virus (Sistemik) 50%, Infeksi Jaringan Lunak (Selulitis) 70%',
  'Angina Pekrotis 80%, Infark Miokard Akut (Serangan Jantung) 80%',
  'Asma 100%, Bronkitis Akut 70%, Penyakit Paru Obstruktif Kronik 50%',
  'Bronkitis Akut 100%',
  'Bronkitis Akut 80%, Asma Bronkial 50%',
  'Dispepsia 100%, Pneumonia 50%, Infeksi Saluran Pernapasan 50%',
  'Dispepsia 80%, refluk Gastroesofagus (GERD) 90%',
  'Gastroenteritis Akut 70%, Infeksi saluran pencernaan 50%',
  'Batu Ginjal 70%, Kolik Renal 80%, Ganguan Saluran kemih 100%, Ureterolitiasis 70%',
  'Peritonitis 50%, Infeksi Saluran Pencernaan Akut 70%, Obstruksi Usus 70%',
  'Kolesistitis Akut 70%, Dispepsia 70%',
  'Appendisitis Akut 70%, Kolika Abdominalis 80%',
  'Dispepsia 80%, Infeksi Saluran Pernapasan Atas 80%',
  'Infeksi Saluran pernapasan atas 70%, Influenza 80%',
  'Infeksi Saluran Pernapasan

In [6]:
eval_df = df[['No', 'Question']]

for k, v in answer_lists.items():
    if k == 'Dr': continue

    print(f'k={k}')
    P, R, F1 = score(refs=answer_lists['Dr'], cands=answer_lists[k], lang='en', verbose=True)
    eval_df[f'{k}_P'] = P.tolist()
    eval_df[f'{k}_R'] = R.tolist()
    eval_df[f'{k}_F1'] = F1.tolist()


k=Claude


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:11<00:00,  5.81s/it]


computing greedy matching.


...

calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:21<00:00, 10.96s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 52.51it/s]


done in 21.95 seconds, 2.05 sentences/sec


In [7]:
eval_df.describe()

,No,Claude_P,Claude_R,Claude_F1,Qwen_P,Qwen_R,Qwen_F1,GPT_P,GPT_R,GPT_F1,Deepseek RAG_P,Deepseek RAG_R,Deepseek RAG_F1,Deepseek non RAG_P,Deepseek non RAG_R,Deepseek non RAG_F1
count,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000
mean,23.000000,0.856848,0.908259,0.881591,0.808401,0.835514,0.821410,0.816437,0.855390,0.835200,0.808853,0.854358,0.830666,0.793207,0.830242,0.810962
std,13.133926,0.043894,0.025749,0.034132,0.035417,0.040780,0.034499,0.039830,0.039013,0.036766,0.039609,0.031684,0.032376,0.040971,0.026859,0.030747
min,1.000000,0.748749,0.846060,0.796143,0.730712,0.756705,0.757629,0.708608,0.754086,0.751015,0.719105,0.782974,0.763067,0.723283,0.767381,0.751236
25%,12.000000,0.832780,0.899328,0.868169,0.789274,0.808681,0.801019,0.788427,0.829779,0.806599,0.780109,0.834833,0.807060,0.766470,0.815271,0.790469
50%,23.000000,0.865030,0.912973,0.884388,0.809001,0.832467,0.826844,0.824803,0.858742,0.838904,0.811930,0.854831,0.828900,0.789994,0.832200,0.807447
75%,34.000000,0.883578,0.927474,0.905364,0.830963,0.867910,0.842796,0.843696,0.876618,0.861037,0.841838,0.877455,0.855788,0.814962,0.848682,0.830417
max,45.000000,0.944525,0.959008,0.951711,0.879325,0.922244,0.899288,0.883068,0.921800,0.899019,0.907079,0.914508,0.893472,0.912450,0.885356,0.898699
